In [2]:

import os
import azureml.core
from azureml.core import Workspace, Dataset, Datastore, ComputeTarget, Experiment, ScriptRunConfig
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline

In [3]:
from azureml.core import Workspace, Datastore, Experiment
ws = Workspace('bd04922c-a444-43dc-892f-74d5090f8a9a','mlplayarearg','testdeployment')
datastore = Datastore.get(ws,'gen2asmanagedidentity')

In [4]:
mydataset = Dataset.File.from_files([(datastore, 'somedata/')])

In [53]:
mydataset.to_path()

['/20201212/file.csv', '/mydata.csv', '/testdataatroot.csv']

In [3]:
pip install -U azureml-sdk

     |████████████████████████████████| 2.1 MB 10.7 MB/s eta 0:00:01
     |████████████████████████████████| 107 kB 65.2 MB/s eta 0:00:01
     |████████████████████████████████| 17.3 MB 58.8 MB/s eta 0:00:01
     |████████████████████████████████| 28.2 MB 89 kB/s s eta 0:00:01
     |████████████████████████████████| 306 kB 53.0 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 971 kB/s  eta 0:00:01
     |████████████████████████████████| 8.6 MB 67.8 MB/s eta 0:00:01
     |████████████████████████████████| 185 kB 65.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 55.1 MB/s eta 0:00:01
     |████████████████████████████████| 7.9 MB 60.6 MB/s eta 0:00:01
ERROR: azureml-widgets 1.5.0 has requirement azureml-core~=1.5.0, but you'll have azureml-core 1.18.0 which is incompatible.
ERROR: azureml-widgets 1.5.0 has requirement azureml-telemetry~=1.5.0, but you'll have azureml-telemetry 1.18.0 which is incompatible.
ERROR: azureml-train-automl 1.5.0 has requirement

In [7]:
pip list

Package                               Version            
------------------------------------- -------------------
absl-py                               0.9.0              
adal                                  1.2.3              
adlfs                                 0.5.5              
aiohttp                               3.6.2              
alabaster                             0.7.12             
alembic                               1.4.2              
anaconda-client                       1.7.2              
anaconda-project                      0.8.3              
ansiwrap                              0.8.4              
antlr4-python3-runtime                4.7.2              
appdirs                               1.4.4              
applicationinsights                   0.11.9             
argcomplete                           1.11.1             
asn1crypto                            1.0.1              
astor                                 0.8.1              
astroid       

In [5]:

from azureml.data import OutputFileDatasetConfig

# learn more about the output config
help(OutputFileDatasetConfig)


Help on class OutputFileDatasetConfig in module azureml.data.output_dataset_config:

class OutputFileDatasetConfig(OutputDatasetConfig)
 |  .. note::    This is an experimental class, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.
 |  
 |  Represent how to copy the output of a run and be promoted as a FileDataset.
 |  
 |  The OutputFileDatasetConfig allows you to specify how you want a particular local path on the compute target
 |  to be uploaded to the specified destination. If no arguments are passed to the constructor, we will
 |  automatically generate a name, a destination, and a local path.
 |  
 |  An example of not passing any arguments:
 |  
 |  .. code-block:: python
 |  
 |      workspace = Workspace.from_config()
 |      experiment = Experiment(workspace, 'output_example')
 |  
 |      output = OutputFileDatasetConfig()
 |  
 |      script_run_config = ScriptRunConfig('.', 'train.py', arguments=[output])
 |  
 |      run = ex

In [6]:
%%writefile step1.py
import os
import sys
import pandas as pd
mounted_input_path = sys.argv[1]
mounted_output_path = sys.argv[2]
os.makedirs(mounted_output_path, exist_ok=True)

print(sys.argv[1])
print(sys.argv[2])
mydf1 = pd.read_csv(os.path.join(sys.argv[1],'mydata.csv'))

mydf2 = pd.read_csv(os.path.join(sys.argv[1],'testdataatroot.csv'))
print(mydf1.shape)
print(mydf2.shape)

test = pd.DataFrame(range(1,1000), columns=["testColumn"])
mounted_output_path = sys.argv[2]
print(f"mounted output path is {mounted_output_path}")
os.makedirs(mounted_output_path, exist_ok=True)

csvfilepath = os.path.join(sys.argv[2] ,'file.csv')
test.to_csv(csvfilepath)




Overwriting step1.py


In [7]:

outputds = OutputFileDatasetConfig(destination=(datastore, 'somedata/20201218'))

WARNING - Class OutputFileDatasetConfig: This is an experimental class, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.
WARNING - Class OutputDatasetConfig: This is an experimental class, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.


In [8]:
prep_step = PythonScriptStep(name='demo step',
                             script_name="step1.py",
                             # mount fashion_ds dataset to the compute_target
                             arguments=[mydataset.as_named_input('myds').as_mount(), outputds],
                             compute_target='automlmanishds3',
                             allow_reuse=True)
pipeline = Pipeline(workspace=ws, steps=[prep_step])

In [9]:
experiment_name = 'showcasing-outputdata'
source_directory  = '.'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
showcasing-outputdata,testdeployment,Link to Azure Machine Learning studio,Link to Documentation


In [10]:
run = experiment.submit(pipeline)
#run.wait_for_completion(show_output=True)

Created step demo step [123402ac][ffc5d589-0b84-4f21-b05b-e26a5f1f1a6f], (This step will run and generate new outputs)
Submitted PipelineRun 927f2ae1-7825-4155-929e-e132c22b0d00
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/showcasing-outputdata/runs/927f2ae1-7825-4155-929e-e132c22b0d00?wsid=/subscriptions/bd04922c-a444-43dc-892f-74d5090f8a9a/resourcegroups/mlplayarearg/workspaces/testdeployment
